In [17]:
import pickle
import numpy as np
import scipy.sparse as ss
import datetime

#### IMPORT DATA ####

data_source = 'TrainSet2014_3.pkl'
#data_source = 'CompetitionSet2017_3.pkl'

full_dynamic_graph_sparse, unconnected_vertex_pairs, year_start, years_delta = pickle.load(open( data_source, "rb" ) )

NUM_OF_VERTICES = 64719 # number of vertices of the semantic net
NUM_OF_EDGES    = full_dynamic_graph_sparse[:, 0].size

#### BUILD ADJACENCY ####

# The concatenation is used to produce a symmetric adjacency matrix
data_rows = np.concatenate([full_dynamic_graph_sparse[:, 0], full_dynamic_graph_sparse[:, 1]])
data_cols = np.concatenate([full_dynamic_graph_sparse[:, 1], full_dynamic_graph_sparse[:, 0]])
data_ones = np.ones(len(data_rows), np.uint32)

adjM = ss.csr_matrix((data_ones, (data_rows, data_cols)), shape=(NUM_OF_VERTICES, NUM_OF_VERTICES))

#### BUILD DEGREE VECTOR ####

degree_vec = np.asarray(adjM.sum(1)).flatten()

print("Imported", data_source)
print("\nBuilt adjacency matrix with:")
print(" - ", NUM_OF_VERTICES, " vertices")
print(" - ", NUM_OF_EDGES, " edges\n")
print("Found ", np.count_nonzero(degree_vec == 0)," unconnected nodes.")

Imported TrainSet2014_3.pkl

Built adjacency matrix with:
 -  64719  vertices
 -  2278611  edges

Found  27230  unconnected nodes.


# Analyzing the links to be predicted

In [18]:
pred_degree0 = degree_vec[unconnected_vertex_pairs[:,0]]
pred_degree1 = degree_vec[unconnected_vertex_pairs[:,1]]

# Counts how many links between two nodes with k = 0:
k0 = np.count_nonzero((pred_degree0 + pred_degree1) == 0)
# Counts how many links between one nodes with k = 0 and one k > 0:
k1 = np.count_nonzero(pred_degree0 * pred_degree1 == 0) - np.count_nonzero((pred_degree0 + pred_degree1) == 0)
# Counts how many links between two nodes with k > 0:
k2 = np.count_nonzero((pred_degree0 * pred_degree1) > 0)

print("Out of 1.000.000 new links:")
print(k0," are between two vertices with k = 0 -> ordered randomly;")
print(k1," are between one vertex with k = 0 and one with k > 0 -> ordered by preferential attachment;")
print(k2," are between two vertices with k > 0 -> ordered by network based predictions;")

Out of 1.000.000 new links:
265966  are between two vertices with k = 0 -> ordered randomly;
500060  are between one vertex with k = 0 and one with k > 0 -> ordered by preferential attachment;
233974  are between two vertices with k > 0 -> ordered by network based predictions;


# L3 Method

In [19]:
def l3_method(adjM, links_to_score):
    # adjM is assumed to be a sparse matrix
    # links_to_score is a list of pairs of nodes (v1, v2)
    # returns an ordered list of those pairs by their index
    # running from 0 to 999.999
    
    # Building D^-1/2.
    degree_vec = np.asarray(adjM.sum(1)).flatten()
    degree_vec = np.where(degree_vec == 0, 1, degree_vec) # to avoid division by 0
    sqrt_degree_vec    = 1/np.sqrt(degree_vec)
    sqrt_degree_matrix = ss.diags(sqrt_degree_vec, 0)

    # Computing Ã
    adjM_tilde = sqrt_degree_matrix*adjM*sqrt_degree_matrix;
    
    # Select rows and columns of A corresponding to links to score
    rows = np.unique(links_to_score[:,0])
    cols = np.unique(links_to_score[:,1])
    
    # Computing P
    p_matrix = adjM[rows,:]*adjM_tilde*adjM[:,cols];
    
    # Note that p_matrix has dimensions size(rows)*size(rows). As it happens in the data from the
    # competition the indices in unconnected_vertex_pairs actually only go up to size(rows), which means
    # that we can use those same indices to call values from p_matrix. If that were not the case,
    # we would have to implement a dictionary here to correct the indices before calling values from
    # p_matrix. I checked and in the competition data the same thing happens, and so I didn't implement
    # the dictionary.
    
    # Ordering relevannt scores
    score_list = np.array(p_matrix[links_to_score[:,0], links_to_score[:,1]]).flatten()
    sorted_predictions = np.argsort(-1.0*score_list)
    
    return sorted_predictions

In [20]:
print(datetime.datetime.now().time())

print("Computing L3 scores...")

sorted_predictions = l3_method(adjM, unconnected_vertex_pairs)

print("Done!")

print(datetime.datetime.now().time())

16:05:58.015341
Computing L3 scores...
Done!
16:06:35.565568


# AUC (for 2014 -> 2017 predictions)

In [21]:
import matplotlib.pyplot as plt

def calculate_ROC(data_vertex_pairs, data_solution, show_plot = False):
    data_solution = np.array(data_solution)
    data_vertex_pairs_sorted = data_solution[data_vertex_pairs]
    
    xpos=[0]
    ypos=[0]
    ROC_vals=[]
    for ii in range(len(data_vertex_pairs_sorted)):
        if data_vertex_pairs_sorted[ii]==1:
            xpos.append(xpos[-1])
            ypos.append(ypos[-1]+1)
        if data_vertex_pairs_sorted[ii]==0:
            xpos.append(xpos[-1]+1)
            ypos.append(ypos[-1])      
            ROC_vals.append(ypos[-1])
    
    ROC_vals=np.array(ROC_vals)/max(ypos)
    ypos=np.array(ypos)/max(ypos)
    xpos=np.array(xpos)/max(xpos)
    
    if show_plot == True:
        plt.plot(xpos, ypos)
        plt.show()
    
    AUC = sum(ROC_vals)/len(ROC_vals)
    return AUC

In [23]:
with open('TrainSet2014_3_solution.pkl', "rb" ) as pkl_file:
        unconnected_vertex_pairs_solution = pickle.load(pkl_file)
    
AUC = calculate_ROC(sorted_predictions, np.array(unconnected_vertex_pairs_solution))

print('Area Under Curve for Evaluation: ', AUC)

Area Under Curve for Evaluation:  0.6928035870074668


# Exporting a submission file (for 2017 data)

In [ ]:
import json

# Save the results for submission.
submit_file = "test_submission_01.json"

all_idx_list_float=list(map(float, sorted_predictions))
with open(submit_file, "w", encoding="utf8") as json_file:
    json.dump(all_idx_list_float, json_file)
    
print("Solution stored as "+submit_file+".\nLooking forward to your submission.")